###### Limpieza de metadatos en el mes de julio
El fichero mensual que contiene los datos de todos los trayectos del mes de julio de 2018 ha sido generado con multitud de espacios y saltos de línea que dificulta la legibilidad del fichero. Sólo ocurre este mes.
Hago una limpieza previa para retirar estos símbolos.

In [ ]:
!cat 201807_Usage_Bicimad.json | tr -d '\r\n' | tr -s ' ' | sed 's/{ "_id"/\n{ "_id"/g' | less

###### Importación del fichero JSON

In [ ]:
#!python -m pip install --upgrade pip
#pip install iso8601

#Habría que poner todos los install necesarios

In [ ]:
import numpy as np
import pandas as pd
from geojson import Feature, FeatureCollection#, Point
import geojson
from shapely.geometry import Point, Polygon, LineString, MultiPoint
import shapely
import geopandas as gpd
import geojsonio
import iso8601

Exploración del mes de agosto de 2018

In [ ]:
rides = pd.read_json('data/201808_Usage_Bicimad.json',lines=True)

In [ ]:
#Limpieza del campo fecha/hora, contenida en un diccionario superfluo
rides['unplug_hourTime'] = rides['unplug_hourTime'].apply(pd.Series)['$date']

In [ ]:
#Parsing a formato fecha gestionando correctamente el huso horario
rides['unplug_hourTime']=pd.to_datetime(rides['unplug_hourTime'].apply(iso8601.parse_date))

In [ ]:
#Agrupación por día para estudiar cuántos trayectos hay y cuántos disponen de track GPS
rides2=rides[['track','unplug_hourTime']].groupby(pd.DatetimeIndex(rides['unplug_hourTime']).normalize()).count()

In [ ]:
rides2['con_track']=rides2['track']/rides2['unplug_hourTime']
rides2

In [ ]:
import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
N = len(rides2.index)
withTrack = rides2['track'] #Rides with a GPS track
noTrack = (rides2['unplug_hourTime']-rides2['track']) #Rides without a GPS trak
ind = np.arange(N)  
width = 0.5       

plt.figure(figsize=(25,5))
p1 = plt.bar(ind, withTrack, width)
p2 = plt.bar(ind, noTrack, width, bottom=withTrack)

plt.ylabel('Number of rides')
plt.xlabel('August 2018')
plt.title('Total rides per day with or without GPS track')
plt.xticks(ind, rides2.index.day)
plt.yticks(np.arange(0, 14000, 1000))
plt.legend((p1[0], p2[0]), ('With GPS track', 'No track'))
plt.figure(figsize=(80,100))

plt.show()

In [ ]:
#Exporto a csv 
rides2.to_csv('diarios_agosto.csv')

In [ ]:
rides.tail()

In [ ]:
#Hay tipos que aún se pueden mejorar
rides.dtypes

In [ ]:
#Numero de trayectos del mes
len(rides)

In [ ]:
#Número de trayectos con información GPS
rides['track'].count()

In [ ]:
#Limpieza del campo _id, contenido en un diccionario superfluo
rides['_id'] = rides['_id'].apply(pd.Series)['$oid']

In [ ]:
rides.head()

#### Gestion del código postal

In [ ]:
#Trim de los espacios
rides['zip_code']=rides.zip_code.str.strip()

In [ ]:
#Validación de códigos postales españoles (cinco dígitos entre 01000 y 52999). Estos son los vacíos o inválidos.
rides[-rides.zip_code.str.contains('^(?:0[1-9]|[1-4]\d|5[0-2])\d{3}$')]['zip_code'].value_counts()

In [ ]:
#Reemplazar vacíos e inválidos por NaN, para tenerlos identificados
rides['zip_code'].replace('^(?!^(?:0[1-9]|[1-4]\d|5[0-2])\d{3}$).*', np.nan, regex=True, inplace=True)

In [ ]:
#Sólo hay 39 códigos postales aparentemente válidos (no necesariamente existen) con un único trayecto
len(rides[-rides['zip_code'].duplicated(keep=False)])

In [ ]:
import json

In [ ]:
#Paseo de uno de Huesca
json.dumps(rides[-rides['zip_code'].duplicated(keep=False)].iloc[0]['track'])

In [ ]:
#Paseo de un turista de Castrillón
rides[rides['zip_code'].replace(np.nan,'').str.contains('33450')].iloc[1]['track']

In [ ]:
rides[['track','unplug_hourTime']].count()

El número de enganche en el que se coge y deposita la bici es irrelevante para nuestro análisis, y nos deshacemos de él.
Tampoco nos aporta nada el id unívoco pero lo dejo de momento, por si acaso

In [ ]:
rides.drop(['idplug_base', 'idunplug_base'], axis=1, inplace=True)

In [ ]:
len(rides[-rides['zip_code'].duplicated(keep=False)])

In [ ]:
rides['zip_code'].value_counts()

Primeras categorías identificadas

In [ ]:
rides['idunplug_base'].value_counts()

El "grupo de edad" y el "tipo de usuario" podemos considerarlos como categorías porque además tienen pocos valores distintos

In [ ]:
rides[['ageRange','idplug_station','idunplug_station','user_type']]=\
rides[['ageRange','idplug_station','idunplug_station','user_type']].astype('category')

In [ ]:
rides.dtypes

Número de trayectos y duración media por grupo de edad:

In [ ]:
rides[['ageRange','travel_time']].groupby('ageRange').agg(['count','mean'])

In [ ]:
pd.to_timedelta(rides[['ageRange','travel_time']].groupby('ageRange').mean()['travel_time'], unit='s')

In [ ]:
pd.DataFrame.

In [ ]:
import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
FeatureCollection(rides['track'][0][features])

In [ ]:
FeatureCollection(rides['track'][0]['features'])

In [ ]:
my_gdf['geometry'][2].distance(my_gdf['geometry'][30])

In [ ]:
hola=gpd.GeoDataFrame.from_features(rides['track'][0]['features'], crs='WGS84')

In [ ]:
hola['geometry']

In [ ]:
hola

In [ ]:
polygon = Polygon([(0, 0), (1, 1), (1, 0)])

In [ ]:
polygon.length

Con **Overpass_turbo** recupero la forma del parque en OpenStreetMap (OSM) en formato geoJSON

In [ ]:
retiro=gpd.read_file('data/retiro.json')

In [ ]:
retiro.drop(columns=retiro.columns.difference(['name','geometry']), inplace=True)

In [ ]:
retiro

In [ ]:
#Guardo los trayectos sin track GPS para posible uso
ridesNoGPS = rides[rides['track'].isnull()]

In [ ]:
ridesNoGPS.to_csv('data/agosto_noGPS.csv')

In [ ]:
len(ridesNoGPS)

In [ ]:
#Trabajamos sobre los trayectos con itinerario GPS disponible
rides = rides.dropna(subset=['track'])

In [ ]:
len(rides)

In [ ]:
rides.loc[0]['track']

In [ ]:
ridea=gpd.GeoDataFrame.from_features(rides['track'][0]['features'], crs='WGS84')
rideb=gpd.GeoDataFrame.from_features(rides['track'][1]['features'], crs='WGS84')

In [ ]:
august=gpd.GeoSeries()

In [ ]:
for r in rides['track'] :
    geoRide=gpd.GeoDataFrame.from_features(r['features'], crs='WGS84')['geometry']
    print(geoRide)
    august=august.append(geoRide)

In [ ]:
audf=gpd.GeoDataFrame(august)

In [ ]:
audf.drop(0,axis=1,inplace=True)

In [ ]:
puntosagosto=shapely.geometry.mapping(audf['geometry'])

In [ ]:
with open('puntosagosto_parcial.json', 'w') as fp:
    geojson.dump(puntosagosto, fp)

In [ ]:
audf.iloc[7500:9000].plot(figsize=[10,20])

In [ ]:
p1500=gpd.GeoSeries(audf.iloc[7500:9000]['geometry'])

In [ ]:
p1500.iloc[1400]

In [ ]:
retiro.intersects(Point([-3.67979499972222, 40.4191539997222]))

In [ ]:
type(p1500)

In [ ]:
retiro.intersects(p1500).value_counts()

In [ ]:
type(audf.iloc[7500:9000]['geometry'])

In [ ]:
retiropol=retiro['geometry'][0]

#### Ojo aquí, parece que ya sí

In [ ]:
audf.intersects(retiropol).value_counts()

In [ ]:
retiro.contains(Point([-3.683297064857466, 40.41493935]))

In [ ]:
retiro.contains(test['geometry'])

In [ ]:
test['geometry'][1]=Point([-3.683297064857466, 40.41493935])

In [ ]:
retiro.representative_point()

In [ ]:
test.index=np.zeros(test.index.size)
test.index

In [ ]:
retiro

In [ ]:
p1500['geometry'].intersects(retiro)

In [ ]:
test=gpd.GeoDataFrame.from_features([
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Point",
        "coordinates": [
          -3.6764931678771973,
          40.41138847106793
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Point",
        "coordinates": [
          -3.683981895446777,
          40.41467234395794
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Point",
        "coordinates": [
          -3.6872434616088867,
          40.41973670935866
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Point",
        "coordinates": [
          -3.689603805541992,
          40.41475403029547
        ]
      }
    }
  ])

In [ ]:
test

In [ ]:
MultiPoint(ridea['geometry'])

In [ ]:
Polygon(retiro['geometry'][0])

```In [1]: from geojson import Feature, Point, FeatureCollection
In [2]: my_feature = Feature(geometry=Point((1.6432, -19.123)), properties={"country": "Spain"})
In [3]: my_other_feature = Feature(geometry=Point((-80.234, -22.532)), properties={'country': 'Brazil'})
In [4]: collection = FeatureCollection([my_feature, my_other_feature])
In [6]: import geopandas
In [7]: geopandas.GeoDataFrame.from_features(collection['features'])
Out[7]:
  country                 geometry
0   Spain   POINT (1.6432 -19.123)
1  Brazil  POINT (-80.234 -22.532)```

In [ ]:
#Por cada trayecto creo que debo meter la sucesión de puntos como una GeoSeries
rides.loc[96274]['track']

In [ ]:
{
         "type": "Polygon",
         "coordinates": [
             [
                 [100.0, 0.0],
                 [101.0, 0.0],
                 [101.0, 1.0],
                 [100.0, 1.0],
                 [100.0, 0.0]
             ]
         ]
     }